In [1]:
import pandas as pd
import geopandas as gpd
import shapely
from scipy.stats import pearsonr
import numpy as np
from libpysal import weights
import time

from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_classification

In [2]:
nadf = pd.DataFrame(columns = ["data2015", "residential2015","data2016", "residential2016","data2017", 
                     "residential2017","data2018", "residential2018","data2019", "residential2019"])

In [3]:
def clean_it(code):
    data2015 = pd.read_csv(code + "grid2015.csv")
    data2016 = pd.read_csv(code + "grid2016.csv")
    data2017 = pd.read_csv(code + "grid2017.csv")
    data2018 = pd.read_csv(code + "grid2018.csv")
    data2019 = pd.read_csv(code + "grid2019.csv")
    
    data2015.rename(columns = {"p_buildBefore2000": "p_buildBefore2000_g", "p_buildAfter2000": "p_buildAfter2000_g", "p_wester_g" : "p_western_g", "afs_school_b" : "afs_vo_b", "afs_treins_g" : "afs_train_g", "nonWestern_b" : "p_nonWestern_b", "mean_WOZ_b" : "mean_WOZ", "pBenefits_b" : "p_benefits_b", "p_NL_g" : "p_NL", "PHoogste20Inkomen_b" : "PHoogste20Inkomen", "PLaagste40Inkomen_b" : "PLaagste40Inkomen" }, inplace = True)
    data2016.rename(columns = {"p_buildBefore2000": "p_buildBefore2000_g", "p_buildAfter2000": "p_buildAfter2000_g", "p_wester_g" : "p_western_g", "afs_school_b" : "afs_vo_b", "afs_treins_g" : "afs_train_g", "nonWestern_b" : "p_nonWestern_b", "mean_WOZ_b" : "mean_WOZ", "pBenefits_b" : "p_benefits_b", "p_NL_g" : "p_NL", "PHoogste20Inkomen_b" : "PHoogste20Inkomen", "PLaagste40Inkomen_b" : "PLaagste40Inkomen" }, inplace = True)
    data2017.rename(columns = {"p_buildBefore2000": "p_buildBefore2000_g", "p_buildAfter2000": "p_buildAfter2000_g", "p_wester_g" : "p_western_g", "afs_school_b" : "afs_vo_b", "afs_treins_g" : "afs_train_g", "nonWestern_b" : "p_nonWestern_b", "mean_WOZ_b" : "mean_WOZ", "pBenefits_b" : "p_benefits_b", "p_NL_g" : "p_NL", "PHoogste20Inkomen_b" : "PHoogste20Inkomen", "PLaagste40Inkomen_b" : "PLaagste40Inkomen" }, inplace = True)
    data2018.rename(columns = {"p_buildBefore2000": "p_buildBefore2000_g", "p_buildAfter2000": "p_buildAfter2000_g", "p_wester_g" : "p_western_g", "afs_school_b" : "afs_vo_b", "afs_treins_g" : "afs_train_g", "nonWestern_b" : "p_nonWestern_b", "mean_WOZ_b" : "mean_WOZ", "pBenefits_b" : "p_benefits_b", "p_NL_g" : "p_NL", "PHoogste20Inkomen_b" : "PHoogste20Inkomen", "PLaagste40Inkomen_b" : "PLaagste40Inkomen" }, inplace = True)
    data2019.rename(columns = {"p_buildBefore2000": "p_buildBefore2000_g", "p_buildAfter2000": "p_buildAfter2000_g", "p_wester_g" : "p_western_g", "afs_school_b" : "afs_vo_b", "afs_treins_g" : "afs_train_g", "nonWestern_b" : "p_nonWestern_b", "mean_WOZ_b" : "mean_WOZ", "pBenefits_b" : "p_benefits_b", "p_NL_g" : "p_NL", "PHoogste20Inkomen_b" : "PHoogste20Inkomen", "PLaagste40Inkomen_b" : "PLaagste40Inkomen" }, inplace = True)
    
    data2015 = data2015.replace({-99997 : np.NaN, np.inf : np.NaN})
    data2016 = data2016.replace({-99997 : np.NaN, np.inf : np.NaN})
    data2017 = data2017.replace({-99997 : np.NaN, np.inf : np.NaN})
    data2018 = data2018.replace({-99997 : np.NaN, np.inf : np.NaN})
    data2019 = data2019.replace({-99997 : np.NaN, np.inf : np.NaN})
    
    b15 = data2015[data2015.woonfunctie>0]
    b16 = data2016[data2016.woonfunctie>0]
    b17 = data2017[data2017.woonfunctie>0]
    b18 = data2018[data2018.woonfunctie>0]
    b19 = data2019[data2019.woonfunctie>0]
    
    def grid_filler(data, grid_column, buurt_column):
        print(grid_column + " missing before: {}".format(data[grid_column].isna().sum()))
        missing_grid_data = data[grid_column].isna().index.values
        data.loc[missing_grid_data, grid_column] = data.loc[missing_grid_data, buurt_column]
        print(grid_column + " missing after: {}".format(data[grid_column].isna().sum()))
        return data[grid_column]
    
    grid_variables = []
    neighborhood_variables = []
    for var in data2015.columns:
        if var.endswith("_g"):
            grid_variables.append(var)
        elif var.endswith("_b"):
            neighborhood_variables.append(var)

    grid_variables.sort(key=str.lower)
    neighborhood_variables.sort(key=str.lower)
    
    ls = list()
    for i, j in zip(grid_variables, neighborhood_variables):
        ls.append((i,j))
    print(ls)
    
    for i in ls:
        data2015[i[0]] = grid_filler(data2015, i[0], i[1])
        data2016[i[0]] = grid_filler(data2016, i[0], i[1])
        data2017[i[0]] = grid_filler(data2017, i[0], i[1])
        data2018[i[0]] = grid_filler(data2018, i[0], i[1])
        data2019[i[0]] = grid_filler(data2019, i[0], i[1])
        
    var = ['geometry', 'bijeenkomstfunctie',
#        'celfunctie', 
           'gezondheidszorgfunctie', 'industriefunctie',
       'kantoorfunctie', 'logiesfunctie', 'onderwijsfunctie', 'sportfunctie',
       'winkelfunctie', 'woonfunctie', 'oppervlakteVerblijfsobject', 'BU_CODE',
       'y', 'inwoner_g', 'geboorte_g', 'inw_014_g', 'inw_1524_g', 'inw_2544_g',
       'inw_4564_g', 'inw_65_g', 'p_NL', 'p_western_g', 'p_nonWestern_g',
       'p_buyhouses_g', 'p_rentals_g', 'p_socialHousing_g', 'medianIncome_g',
       'avg_electricity_g', 'avg_gas_g', 'p_benefits_g', 'afs_haprak_g',
       'afs_ziek_g', 'afs_superm_g', 'afs_vo_g', 'afs_oprit_g', 'afs_train_g',
       'afs_transit_g', 'p_buildBefore2000_g', 'p_buildAfter2000_g', 'mean_WOZ', 
        'PLaagste40Inkomen', 'PHoogste20Inkomen']
    
    t15 = data2015[var]
    t16 = data2016[var]
    t17 = data2017[var]
    t18 = data2018[var]
    t19 = data2019[var]

    
    t15["n_missing"] = t15.isna().sum(axis=1)
    t16["n_missing"] = t16.isna().sum(axis=1)
    t17["n_missing"] = t17.isna().sum(axis=1)
    t18["n_missing"] = t18.isna().sum(axis=1)
    t19["n_missing"] = t19.isna().sum(axis=1)

    t15b = t15[(t15.woonfunctie > 0) & (t15.n_missing < 15)]
    t16b = t16[(t16.woonfunctie > 0) & (t16.n_missing < 15)]
    t17b = t17[(t17.woonfunctie > 0) & (t17.n_missing < 15)]
    t18b = t18[(t18.woonfunctie > 0) & (t18.n_missing < 15)]
    t19b = t19[(t19.woonfunctie > 0) & (t19.n_missing < 15)]

    t15b = gpd.GeoDataFrame(t15b)
    t15b.geometry = t15b.geometry.apply(shapely.wkt.loads)

    t16b = gpd.GeoDataFrame(t16b)
    t16b.geometry = t16b.geometry.apply(shapely.wkt.loads)

    t17b = gpd.GeoDataFrame(t17b)
    t17b.geometry = t17b.geometry.apply(shapely.wkt.loads)

    t18b = gpd.GeoDataFrame(t18b)
    t18b.geometry = t18b.geometry.apply(shapely.wkt.loads)

    t19b = gpd.GeoDataFrame(t19b)
    t19b.geometry = t19b.geometry.apply(shapely.wkt.loads)
    
    var2 = ['bijeenkomstfunctie', 'geometry',
#        'celfunctie', 
            'gezondheidszorgfunctie', 'industriefunctie',
       'kantoorfunctie', 'logiesfunctie', 'onderwijsfunctie', 'sportfunctie',
       'winkelfunctie', 'woonfunctie', 'oppervlakteVerblijfsobject', 
        'inwoner_g', 'geboorte_g', 'inw_014_g', 'inw_1524_g', 'inw_2544_g',
       'inw_4564_g', 'inw_65_g', 'p_western_g', 'p_nonWestern_g',
       'p_buyhouses_g', 'p_rentals_g', 'p_socialHousing_g', 'medianIncome_g',
       'avg_electricity_g', 'avg_gas_g', 'p_benefits_g', 'afs_haprak_g',
       'afs_ziek_g', 'afs_superm_g', 'afs_vo_g', 'afs_oprit_g', 'afs_train_g',
       'afs_transit_g', 'p_buildBefore2000_g', 'p_buildAfter2000_g', 'mean_WOZ', 
        'PLaagste40Inkomen', 'PHoogste20Inkomen']
    
    t15b = t15b[var2]
    t16b = t16b[var2]
    t17b = t17b[var2]
    t18b = t18b[var2]
    t19b = t19b[var2]
    
    def fill_final_na(data):
        t0 = time.time()
        total_na = data.isna().sum().sum()
        all_geom = data[["geometry", "sportfunctie"]]

        data = data.drop(["geometry"], axis = 1)
        incomplete = data[data.isnull().any(axis=1)]
        complete = gpd.GeoDataFrame(data[~data.isnull().any(axis=1)]) # store the rows that have no na values



        # create list of combinatoins of missing columns
        inc_list = list()
        for _, i in incomplete.iterrows():
            if not tuple(i.index[i.isna()]) in inc_list:
                inc_list.append(tuple(i.index[i.isna()]))

    #   Create weightsmatrix and transform to row_standardized            
    #     w_all = weights.distance.KNN.from_dataframe(all_geom, k = 4, silence_warnings = True)
    #     w_all.transform = 'r'


        columns = data.columns

        # loop over the missing column combinations
        def train_creater(inp, w, train = "train", test = False):
            """
            returns: new dataframe including the neighbors data
            """
            # Get neigbors per index
            neighbors = [ w.neighbors[k] for k in inp.index.values.tolist()]
            if test:

                df = pd.concat([inp, train])

                rows = [ np.concatenate(df.loc[j].values) for j in neighbors] 
                distances = [list(all_geom.loc[neigh].distance(all_geom.loc[df.iloc[i].name].geometry).values) for i, neigh in enumerate(neighbors)]
            else:
                # get rows of the neigbors, tranform to np and concatenate them
                rows = [ np.concatenate(inp.loc[j].values) for j in neighbors] 
                distances = [list(all_geom.loc[neigh].distance(all_geom.loc[inp.iloc[i].name].geometry).values) for i, neigh in enumerate(neighbors)]


            return pd.concat([pd.DataFrame(rows).set_index(inp.index), inp, pd.DataFrame(distances).set_index(inp.index) ], axis=1)

        for i, cols in enumerate(inc_list):
            cols = list(cols)
            x_train = complete.drop(cols, axis = 1) # only keep the cols to trained upon
            y_train = complete[cols] # y has the na values to be predicted

            w_train  = weights.distance.KNN.from_dataframe(all_geom.loc[x_train.index], k = 4, silence_warnings = True)
            w_train.transform = 'r'

            X_train = train_creater(x_train, w_train)

            # Voor x predict only keep the rows where there are only NA's in cols
            pred = incomplete[(incomplete[cols].isna().all(axis = 1) & (incomplete[columns.drop(cols)].notna().all(axis = 1)))]
            X_pred = pred.drop(cols, axis = 1)

            # create prediction weights of missing rows as well as known rows
            w_pred =  weights.distance.KNN.from_dataframe(all_geom.loc[X_train.index.append(X_pred.index)], k = 4, silence_warnings = True)
            w_pred.transform = 'r'
            X_pred = train_creater(X_pred, w_pred, x_train, True)

            # loop over each missing variable in cols and create model per 
            for col in cols:

                clf = RandomForestRegressor(max_depth=6, random_state=0)
                clf.fit(X_train, y_train[col].values.ravel())    
                y_pred = clf.predict(X_pred)
                incomplete.loc[X_pred.index, col] = y_pred

            print("{} na's left of original {}, {:.2f}% of cols done, time spend: {:.0f}".format(incomplete.isna().sum().sum(),  total_na, i / len(inc_list) * 100, time.time() - t0))
        print("total time: {:.0f}".format(time.time() - t0))
        return pd.concat([incomplete, complete])
    
    t15b = fill_final_na(t15b)
    t16b = fill_final_na(t16b)
    t17b = fill_final_na(t17b)
    t18b = fill_final_na(t18b)
    t19b = fill_final_na(t19b)
    
    t15b = t15b.join(data2015[["C28992R100", "geometry", "y", "BU_CODE"]])
    t16b = t16b.join(data2016[["C28992R100", "geometry", "y", "BU_CODE"]])
    t17b = t17b.join(data2017[["C28992R100", "geometry", "y", "BU_CODE"]])
    t18b = t18b.join(data2018[["C28992R100", "geometry", "y", "BU_CODE"]])
    t19b = t19b.join(data2019[["C28992R100", "geometry", "y", "BU_CODE"]])
    
    t15b.to_csv("../Data/filled/2015/" + code + "filled_data15.csv")
    t16b.to_csv("../Data/filled/2016/" + code + "filled_data16.csv")
    t17b.to_csv("../Data/filled/2017/" + code + "filled_data17.csv")
    t18b.to_csv("../Data/filled/2018/" + code + "filled_data18.csv")
    t19b.to_csv("../Data/filled/2019/" + code + "filled_data19.csv")

In [4]:
clean_it("EH")

[('afs_haprak_g', 'afs_huisarts_b'), ('afs_oprit_g', 'afs_oprit_b'), ('afs_superm_g', 'afs_supermarkt_b'), ('afs_train_g', 'afs_train_b'), ('afs_transit_g', 'afs_transit_b'), ('afs_vo_g', 'afs_vo_b'), ('afs_ziek_g', 'afs_ziekenhuis_b'), ('avg_electricity_g', 'avg_electicity_b'), ('avg_gas_g', 'avg_gas_b'), ('geboorte_g', 'birth_b'), ('inw_014_g', 'inw14_b'), ('inw_1524_g', 'inw24_b'), ('inw_2544_g', 'inw44_b'), ('inw_4564_g', 'inw64_b'), ('inw_65_g', 'inw65_b'), ('inwoner_g', 'Inwoners_b'), ('medianIncome_g', 'medianInkomen_b'), ('p_benefits_g', 'p_benefits_b'), ('p_buildAfter2000_g', 'p_buildAfter2000_b'), ('p_buildBefore2000_g', 'p_buildBefore2000_b'), ('p_buyhouses_g', 'p_buyHouses_b'), ('p_nonWestern_g', 'p_nonWestern_b'), ('p_rentals_g', 'p_rental_houses_b'), ('p_socialHousing_g', 'p_socialHousing_b'), ('p_western_g', 'western_b')]
afs_haprak_g missing before: 9518
afs_haprak_g missing after: 7379
afs_haprak_g missing before: 9511
afs_haprak_g missing after: 7379
afs_haprak_g miss

C:\Users\Gebruiker\anaconda3\lib\site-packages\ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Gebruiker\anaconda3\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Gebruiker\anaconda3\lib\site-packages\ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

3373 na's left of original 3773, 0.00% of cols done, time spend: 47


C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

3361 na's left of original 3773, 9.09% of cols done, time spend: 117


...

C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

154 na's left of original 25134, 85.71% of cols done, time spend: 126


C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

0 na's left of original 25134, 92.86% of cols done, time spend: 135
total time: 135


KeyError: "['C28992R100'] not in index"

In [3]:
data2015 = pd.read_csv("BRgrid2015.csv")
data2016 = pd.read_csv("BRgrid2016.csv")
data2017 = pd.read_csv("BRgrid2017.csv")
data2018 = pd.read_csv("BRgrid2018.csv")
data2019 = pd.read_csv("BRgrid2019.csv")

In [4]:
data2015.rename(columns = {"p_buildBefore2000": "p_buildBefore2000_g", "p_buildAfter2000": "p_buildAfter2000_g", "p_wester_g" : "p_western_g", "afs_school_b" : "afs_vo_b", "afs_treins_g" : "afs_train_g", "nonWestern_b" : "p_nonWestern_b", "mean_WOZ_b" : "mean_WOZ", "pBenefits_b" : "p_benefits_b", "p_NL_g" : "p_NL", "PHoogste20Inkomen_b" : "PHoogste20Inkomen", "PLaagste40Inkomen_b" : "PLaagste40Inkomen" }, inplace = True)
data2016.rename(columns = {"p_buildBefore2000": "p_buildBefore2000_g", "p_buildAfter2000": "p_buildAfter2000_g", "p_wester_g" : "p_western_g", "afs_school_b" : "afs_vo_b", "afs_treins_g" : "afs_train_g", "nonWestern_b" : "p_nonWestern_b", "mean_WOZ_b" : "mean_WOZ", "pBenefits_b" : "p_benefits_b", "p_NL_g" : "p_NL", "PHoogste20Inkomen_b" : "PHoogste20Inkomen", "PLaagste40Inkomen_b" : "PLaagste40Inkomen" }, inplace = True)
data2017.rename(columns = {"p_buildBefore2000": "p_buildBefore2000_g", "p_buildAfter2000": "p_buildAfter2000_g", "p_wester_g" : "p_western_g", "afs_school_b" : "afs_vo_b", "afs_treins_g" : "afs_train_g", "nonWestern_b" : "p_nonWestern_b", "mean_WOZ_b" : "mean_WOZ", "pBenefits_b" : "p_benefits_b", "p_NL_g" : "p_NL", "PHoogste20Inkomen_b" : "PHoogste20Inkomen", "PLaagste40Inkomen_b" : "PLaagste40Inkomen" }, inplace = True)
data2018.rename(columns = {"p_buildBefore2000": "p_buildBefore2000_g", "p_buildAfter2000": "p_buildAfter2000_g", "p_wester_g" : "p_western_g", "afs_school_b" : "afs_vo_b", "afs_treins_g" : "afs_train_g", "nonWestern_b" : "p_nonWestern_b", "mean_WOZ_b" : "mean_WOZ", "pBenefits_b" : "p_benefits_b", "p_NL_g" : "p_NL", "PHoogste20Inkomen_b" : "PHoogste20Inkomen", "PLaagste40Inkomen_b" : "PLaagste40Inkomen" }, inplace = True)
data2019.rename(columns = {"p_buildBefore2000": "p_buildBefore2000_g", "p_buildAfter2000": "p_buildAfter2000_g", "p_wester_g" : "p_western_g", "afs_school_b" : "afs_vo_b", "afs_treins_g" : "afs_train_g", "nonWestern_b" : "p_nonWestern_b", "mean_WOZ_b" : "mean_WOZ", "pBenefits_b" : "p_benefits_b", "p_NL_g" : "p_NL", "PHoogste20Inkomen_b" : "PHoogste20Inkomen", "PLaagste40Inkomen_b" : "PLaagste40Inkomen" }, inplace = True)

In [5]:
data2015 = data2015.replace({-99997 : np.NaN, np.inf : np.NaN})
data2016 = data2016.replace({-99997 : np.NaN, np.inf : np.NaN})
data2017 = data2017.replace({-99997 : np.NaN, np.inf : np.NaN})
data2018 = data2018.replace({-99997 : np.NaN, np.inf : np.NaN})
data2019 = data2019.replace({-99997 : np.NaN, np.inf : np.NaN})


In [6]:
b15 = data2015[data2015.woonfunctie>0]
b16 = data2016[data2016.woonfunctie>0]
b17 = data2017[data2017.woonfunctie>0]
b18 = data2018[data2018.woonfunctie>0]
b19 = data2019[data2019.woonfunctie>0]

In [7]:
nadf["data2015"] = data2015.isna().sum() / len(data2015)
nadf["residential2015"] = b15.isna().sum() / len(b15)

nadf["data2016"] = data2016.isna().sum() / len(data2015)
nadf["residential2016"] = b16.isna().sum() / len(b15)

nadf["data2017"] = data2017.isna().sum() / len(data2015)
nadf["residential2017"] = b17.isna().sum() / len(b15)

nadf["data2018"] = data2018.isna().sum() / len(data2015)
nadf["residential2018"] = b18.isna().sum() / len(b15)

nadf["data2019"] = data2019.isna().sum() / len(data2015)
nadf["residential2019"] = b19.isna().sum() / len(b15)

In [8]:
def grid_filler(data, grid_column, buurt_column):
    print(grid_column + " missing before: {}".format(data[grid_column].isna().sum()))
    missing_grid_data = data[grid_column].isna().index.values
    data.loc[missing_grid_data, grid_column] = data.loc[missing_grid_data, buurt_column]
    print(grid_column + " missing after: {}".format(data[grid_column].isna().sum()))
    return data[grid_column]

In [9]:
grid_variables = []
neighborhood_variables = []
for var in data2015.columns:
    if var.endswith("_g"):
        grid_variables.append(var)
    elif var.endswith("_b"):
        neighborhood_variables.append(var)
        
grid_variables.sort(key=str.lower)
neighborhood_variables.sort(key=str.lower)

In [10]:
ls = list()
for i, j in zip(grid_variables, neighborhood_variables):
    ls.append((i,j))
print(ls)

[('afs_haprak_g', 'afs_huisarts_b'), ('afs_oprit_g', 'afs_oprit_b'), ('afs_superm_g', 'afs_supermarkt_b'), ('afs_train_g', 'afs_train_b'), ('afs_transit_g', 'afs_transit_b'), ('afs_vo_g', 'afs_vo_b'), ('afs_ziek_g', 'afs_ziekenhuis_b'), ('avg_electricity_g', 'avg_electicity_b'), ('avg_gas_g', 'avg_gas_b'), ('geboorte_g', 'birth_b'), ('inw_014_g', 'inw14_b'), ('inw_1524_g', 'inw24_b'), ('inw_2544_g', 'inw44_b'), ('inw_4564_g', 'inw64_b'), ('inw_65_g', 'inw65_b'), ('inwoner_g', 'Inwoners_b'), ('medianIncome_g', 'medianInkomen_b'), ('p_benefits_g', 'p_benefits_b'), ('p_buildAfter2000_g', 'p_buildAfter2000_b'), ('p_buildBefore2000_g', 'p_buildBefore2000_b'), ('p_buyhouses_g', 'p_buyHouses_b'), ('p_nonWestern_g', 'p_nonWestern_b'), ('p_rentals_g', 'p_rental_houses_b'), ('p_socialHousing_g', 'p_socialHousing_b'), ('p_western_g', 'western_b')]


In [11]:
for i in ls:
    data2015[i[0]] = grid_filler(data2015, i[0], i[1])
    data2016[i[0]] = grid_filler(data2016, i[0], i[1])
    data2017[i[0]] = grid_filler(data2017, i[0], i[1])
    data2018[i[0]] = grid_filler(data2018, i[0], i[1])
    data2019[i[0]] = grid_filler(data2019, i[0], i[1])
    

afs_haprak_g missing before: 15560
afs_haprak_g missing after: 12807
afs_haprak_g missing before: 15546
afs_haprak_g missing after: 12807
afs_haprak_g missing before: 15518
afs_haprak_g missing after: 12807
afs_haprak_g missing before: 15510
afs_haprak_g missing after: 12807
afs_haprak_g missing before: 19324
afs_haprak_g missing after: 12807
afs_oprit_g missing before: 15555
afs_oprit_g missing after: 12807
afs_oprit_g missing before: 15537
afs_oprit_g missing after: 12807
afs_oprit_g missing before: 15518
afs_oprit_g missing after: 12807
afs_oprit_g missing before: 15510
afs_oprit_g missing after: 12807
afs_oprit_g missing before: 19324
afs_oprit_g missing after: 12807
afs_superm_g missing before: 15555
afs_superm_g missing after: 12807
afs_superm_g missing before: 15537
afs_superm_g missing after: 12807
afs_superm_g missing before: 15518
afs_superm_g missing after: 12807
afs_superm_g missing before: 15510
afs_superm_g missing after: 12807
afs_superm_g missing before: 19324
afs_super

In [12]:
def visualize_NANs(data):
    data["n_missing"] = data.isna().sum(axis=1)
    data.plot(figsize = (40,40), alpha=0.5, edgecolor = "b", column='n_missing',legend=True)

In [13]:
var = ['geometry', 'bijeenkomstfunctie',
       'celfunctie', 'gezondheidszorgfunctie', 'industriefunctie',
       'kantoorfunctie', 'logiesfunctie', 'onderwijsfunctie', 'sportfunctie',
       'winkelfunctie', 'woonfunctie', 'oppervlakteVerblijfsobject', 'BU_CODE',
       'y', 'inwoner_g', 'geboorte_g', 'inw_014_g', 'inw_1524_g', 'inw_2544_g',
       'inw_4564_g', 'inw_65_g', 'p_NL', 'p_western_g', 'p_nonWestern_g',
       'p_buyhouses_g', 'p_rentals_g', 'p_socialHousing_g', 'medianIncome_g',
       'avg_electricity_g', 'avg_gas_g', 'p_benefits_g', 'afs_haprak_g',
       'afs_ziek_g', 'afs_superm_g', 'afs_vo_g', 'afs_oprit_g', 'afs_train_g',
       'afs_transit_g', 'p_buildBefore2000_g', 'p_buildAfter2000_g', 'mean_WOZ', 
        'PLaagste40Inkomen', 'PHoogste20Inkomen']

In [14]:
t15 = data2015[var]
t16 = data2016[var]
t17 = data2017[var]
t18 = data2018[var]
t19 = data2019[var]


In [15]:
t15["n_missing"] = t15.isna().sum(axis=1)
t16["n_missing"] = t16.isna().sum(axis=1)
t17["n_missing"] = t17.isna().sum(axis=1)
t18["n_missing"] = t18.isna().sum(axis=1)
t19["n_missing"] = t19.isna().sum(axis=1)

t15b = t15[(t15.woonfunctie > 0) & (t15.n_missing < 15)]
t16b = t16[(t16.woonfunctie > 0) & (t16.n_missing < 15)]
t17b = t17[(t17.woonfunctie > 0) & (t17.n_missing < 15)]
t18b = t18[(t18.woonfunctie > 0) & (t18.n_missing < 15)]
t19b = t19[(t19.woonfunctie > 0) & (t19.n_missing < 15)]

t15b = gpd.GeoDataFrame(t15b)
t15b.geometry = t15b.geometry.apply(shapely.wkt.loads)

t16b = gpd.GeoDataFrame(t16b)
t16b.geometry = t16b.geometry.apply(shapely.wkt.loads)

t17b = gpd.GeoDataFrame(t17b)
t17b.geometry = t17b.geometry.apply(shapely.wkt.loads)

t18b = gpd.GeoDataFrame(t18b)
t18b.geometry = t18b.geometry.apply(shapely.wkt.loads)

t19b = gpd.GeoDataFrame(t19b)
t19b.geometry = t19b.geometry.apply(shapely.wkt.loads)


C:\Users\Gebruiker\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Gebruiker\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Gebruiker\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [16]:
data2015.columns

Index(['C28992R100', 'Unnamed: 0', 'geometry', 'bijeenkomstfunctie',
       'celfunctie', 'gezondheidszorgfunctie', 'industriefunctie',
       'kantoorfunctie', 'logiesfunctie', 'onderwijsfunctie', 'sportfunctie',
       'winkelfunctie', 'woonfunctie', 'oppervlakteVerblijfsobject', 'BU_CODE',
       'y', 'inwoner_g', 'geboorte_g', 'inw_014_g', 'inw_1524_g', 'inw_2544_g',
       'inw_4564_g', 'inw_65_g', 'p_NL', 'p_western_g', 'p_nonWestern_g',
       'p_buyhouses_g', 'p_rentals_g', 'p_socialHousing_g', 'medianIncome_g',
       'avg_electricity_g', 'avg_gas_g', 'p_benefits_g', 'afs_haprak_g',
       'afs_ziek_g', 'afs_superm_g', 'afs_vo_g', 'afs_oprit_g', 'afs_train_g',
       'afs_transit_g', 'p_buildBefore2000_g', 'p_buildAfter2000_g',
       'afs_huisarts_b', 'afs_ziekenhuis_b', 'afs_supermarkt_b', 'afs_vo_b',
       'afs_oprit_b', 'afs_train_b', 'afs_transit_b', 'Inwoners_b', 'inw14_b',
       'inw24_b', 'inw44_b', 'inw64_b', 'inw65_b', 'western_b',
       'p_nonWestern_b', 'birth_b

In [17]:
var2 = ['bijeenkomstfunctie', 'geometry',
       'celfunctie', 'gezondheidszorgfunctie', 'industriefunctie',
       'kantoorfunctie', 'logiesfunctie', 'onderwijsfunctie', 'sportfunctie',
       'winkelfunctie', 'woonfunctie', 'oppervlakteVerblijfsobject', 
        'inwoner_g', 'geboorte_g', 'inw_014_g', 'inw_1524_g', 'inw_2544_g',
       'inw_4564_g', 'inw_65_g', 'p_western_g', 'p_nonWestern_g',
       'p_buyhouses_g', 'p_rentals_g', 'p_socialHousing_g', 'medianIncome_g',
       'avg_electricity_g', 'avg_gas_g', 'p_benefits_g', 'afs_haprak_g',
       'afs_ziek_g', 'afs_superm_g', 'afs_vo_g', 'afs_oprit_g', 'afs_train_g',
       'afs_transit_g', 'p_buildBefore2000_g', 'p_buildAfter2000_g', 'mean_WOZ', 
        'PLaagste40Inkomen', 'PHoogste20Inkomen']

In [18]:
t15b = t15b[var2]
t15b = gpd.GeoDataFrame(t15b)
t16b = t16b[var2]
t16b = gpd.GeoDataFrame(t16b)
t17b = t17b[var2]
t17b = gpd.GeoDataFrame(t17b)
t18b = t18b[var2]
t18b = gpd.GeoDataFrame(t18b)
t19b = t19b[var2]
t19b = gpd.GeoDataFrame(t19b)



In [19]:
def fill_final_na(data):
    t0 = time.time()
    total_na = data.isna().sum().sum()
    all_geom = data[["geometry", "sportfunctie"]]

    data = data.drop(["geometry"], axis = 1)
    incomplete = data[data.isnull().any(axis=1)]
    complete = gpd.GeoDataFrame(data[~data.isnull().any(axis=1)]) # store the rows that have no na values
    
    
    
    # create list of combinatoins of missing columns
    inc_list = list()
    for _, i in incomplete.iterrows():
        if not tuple(i.index[i.isna()]) in inc_list:
            inc_list.append(tuple(i.index[i.isna()]))
            
#   Create weightsmatrix and transform to row_standardized            
#     w_all = weights.distance.KNN.from_dataframe(all_geom, k = 4, silence_warnings = True)
#     w_all.transform = 'r'
    
    
    columns = data.columns
    
    # loop over the missing column combinations
    def train_creater(inp, w, train = "train", test = False):
        """
        returns: new dataframe including the neighbors data
        """
        # Get neigbors per index
        neighbors = [ w.neighbors[k] for k in inp.index.values.tolist()]
        if test:
            
            df = pd.concat([inp, train])
            
            rows = [ np.concatenate(df.loc[j].values) for j in neighbors] 
            distances = [list(all_geom.loc[neigh].distance(all_geom.loc[df.iloc[i].name].geometry).values) for i, neigh in enumerate(neighbors)]
        else:
            # get rows of the neigbors, tranform to np and concatenate them
            rows = [ np.concatenate(inp.loc[j].values) for j in neighbors] 
            distances = [list(all_geom.loc[neigh].distance(all_geom.loc[inp.iloc[i].name].geometry).values) for i, neigh in enumerate(neighbors)]
        
        
        return pd.concat([pd.DataFrame(rows).set_index(inp.index), inp, pd.DataFrame(distances).set_index(inp.index) ], axis=1)
        
    for i, cols in enumerate(inc_list):
        cols = list(cols)
        x_train = complete.drop(cols, axis = 1) # only keep the cols to trained upon
        y_train = complete[cols] # y has the na values to be predicted
       
        w_train  = weights.distance.KNN.from_dataframe(all_geom.loc[x_train.index], k = 4, silence_warnings = True)
        w_train.transform = 'r'
        
        X_train = train_creater(x_train, w_train)
        
        # Voor x predict only keep the rows where there are only NA's in cols
        pred = incomplete[(incomplete[cols].isna().all(axis = 1) & (incomplete[columns.drop(cols)].notna().all(axis = 1)))]
        X_pred = pred.drop(cols, axis = 1)

        # create prediction weights of missing rows as well as known rows
        w_pred =  weights.distance.KNN.from_dataframe(all_geom.loc[X_train.index.append(X_pred.index)], k = 4, silence_warnings = True)
        w_pred.transform = 'r'
        X_pred = train_creater(X_pred, w_pred, x_train, True)
        
        # loop over each missing variable in cols and create model per 
        for col in cols:
            
            clf = RandomForestRegressor(max_depth=6, random_state=0)
            clf.fit(X_train, y_train[col].values.ravel())    
            y_pred = clf.predict(X_pred)
            incomplete.loc[X_pred.index, col] = y_pred
            
        print("{} na's left of original {}, {:.2f}% of cols done, time spend: {:.0f}".format(incomplete.isna().sum().sum(),  total_na, i / len(inc_list) * 100, time.time() - t0))
    print("total time: {:.0f}".format(time.time() - t0))
    return pd.concat([incomplete, complete])
            


In [20]:

t15b = fill_final_na(t15b)
t16b = fill_final_na(t16b)
t17b = fill_final_na(t17b)
t18b = fill_final_na(t18b)
t19b = fill_final_na(t19b)

C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


788 na's left of original 885, 0.00% of cols done, time spend: 12


C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

592 na's left of original 885, 14.29% of cols done, time spend: 38


C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


...

C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

180 na's left of original 2409, 75.00% of cols done, time spend: 84


C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

0 na's left of original 2409, 87.50% of cols done, time spend: 109
total time: 109


C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [117]:
t15b

,bijeenkomstfunctie,celfunctie,gezondheidszorgfunctie,industriefunctie,kantoorfunctie,logiesfunctie,onderwijsfunctie,sportfunctie,winkelfunctie,woonfunctie,...,afs_superm_g,afs_vo_g,afs_oprit_g,afs_train_g,afs_transit_g,p_buildBefore2000_g,p_buildAfter2000_g,mean_WOZ,PLaagste40Inkomen,PHoogste20Inkomen
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,3.3,4.3,2.1,5.7,7.9,94.0,6.0,391.764651,34.453318,37.313453
345,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,3.3,4.3,2.1,5.7,7.9,94.0,6.0,393.048448,34.618565,38.903016
349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,3.3,4.3,2.1,5.7,7.9,94.0,6.0,390.136723,34.509750,38.094073
611,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,3.2,3.5,1.4,4.4,6.5,88.0,12.0,283.960709,35.672029,35.528626
625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,3.2,3.5,1.4,4.4,6.5,88.0,12.0,284.419384,35.615597,37.537007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,...,0.7,1.4,0.9,1.3,7.3,92.0,8.0,189.000000,37.100000,23.800000
13496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,...,0.7,1.4,0.9,1.3,7.3,92.0,8.0,189.000000,37.100000,23.800000
13497,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,17.0,...,0.7,1.4,0.9,1.3,7.3,92.0,8.0,189.000000,37.100000,23.800000
13498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,...,0.7,1.4,0.9,1.3,7.3,92.0,8.0,189.000000,37.100000,23.800000


In [28]:
t15b = t15b.join(data2015[["C28992R100", "geometry", "y", "BU_CODE"]])
t16b = t16b.join(data2016[["C28992R100", "geometry", "y", "BU_CODE"]])
t17b = t17b.join(data2017[["C28992R100", "geometry", "y", "BU_CODE"]])
t18b = t18b.join(data2018[["C28992R100", "geometry", "y", "BU_CODE"]])
t19b = t19b.join(data2019[["C28992R100", "geometry", "y", "BU_CODE"]])

In [29]:
t15b.to_csv("BRfilled_data15.csv")
t16b.to_csv("BRfilled_data16.csv")
t17b.to_csv("BRfilled_data17.csv")
t18b.to_csv("BRfilled_data18.csv")
t19b.to_csv("BRfilled_data19.csv")

In [26]:
t15b.shape

(4404, 39)